## Outlierdetectie met **Kleuren**afbeeldingen
### Is het een bos, een woestijn of een meer?

Vandaag gaan we aan de slag met **kleurenafbeeldingen** voor outlierdetectie. Specifiek zullen we:

1. Een **autoencoder** gebruiken voor feature-extractie en het detecteren van afwijkingen.
2. Data categoriseren en outliers detecteren met behulp van **KNN (k-Nearest Neighbors)** en **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**.

Dit bouwt voort op ons eerdere werk, met de extra uitdaging van het werken met RGB (kleur) waarden in afbeeldingen.

---

### **Stappenplan**

#### **Stap 0**: Importeer de datasets
- Download en importeer de datasets die beschikbaar zijn gesteld via Teams.

#### **Stap 1**: Herkennen en gebruiken van kleurkanalen
- Begrijp en haal de verschillende **kleurkanalen** (Rood, Groen, Blauw) uit de afbeeldingen.
- Verken hoe deze kanalen bijdragen aan de algemene kenmerken van de afbeeldingen.
- Train een auto encoder alleen op de blauw waardes van de lake images.
- Check hoe deze werkt met een aantal test images (waar je dus ook alleen het blauw kanaal van pakt)
    -   Deze test dataset moet ook een aantal bos / woestijn fotos bevatten




In [2]:
import os
os.listdir()

['.DS_Store',
 'autoencoder_output.png',
 'autoencoder_output_2.png',
 'autoencoder_RGB.ipynb',
 'Desert',
 'Forest',
 'Mixed',
 'resized_lake_water',
 'Werkcollege_7.pptx',
 'Werkcollege_7_RGB.ipynb']

In [4]:
import cv2
import os
from PIL import Image
import numpy as np
from skimage import io


def import_images(dir):
    image_array = []
    for filename in os.listdir(dir):

        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif')):
            image = io.imread(dir + "/" + filename)
            image_array.append(image)
            
        
    return image_array
            


desert = import_images('Mixed')



In [5]:
image = desert[0]

In [6]:
# Extract the blue channel
blue_channel = image[:, :, 0]  # The first channel in BGR is blue

# Resize the blue channel to a fixed size (optional)
resized_blue_channel = cv2.resize(blue_channel, (128, 128), interpolation=cv2.INTER_LINEAR)



### Stap 2: Kleurenhistogrammen gebruiken om foto's te categoriseren met KNN

In deze stap gebruiken we kleurenhistogrammen om foto's te categoriseren in drie categorieën: **bos**, **woestijn** en **meer**.

Net zoals vorige keer, gaan we een clustering uitvoeren met KNN. Dit keer maken we gebruik van **3 clusters** om de data te groeperen.

Om dit te doen maken we gebruik van de volgende bronnen en de OpenCV-functie `cv2.calcHist`:

- [PyImageSearch: OpenCV Image Histograms](https://pyimagesearch.com/2021/04/28/opencv-image-histograms-cv2-calchist/)
- [GeeksforGeeks: cv2.calcHist Methode](https://www.geeksforgeeks.org/python-opencv-cv2-calchist-method/)

---

### Belangrijke Stappen:

1. **Bereken Kleurenhistogrammen**:
   - De functie `calculate_histogram` berekent de RGB-histogrammen voor elke afbeelding en combineert deze tot één enkele featurevector.

2. **Dimensiereductie met PCA**:
   - PCA reduceert de hoog-dimensionale histogramvectoren tot een lagere-dimensionale representatie (in dit geval 10 componenten).

3. **Clustering met K-Means**:
   - K-means wordt toegepast op de PCA-getransformeerde features om de afbeeldingen in 3 categorieën te clusteren.

4. **Visualisatie**:
   - De eerste twee PCA-componenten worden gevisualiseerd in een scatter plot, waarbij de kleuren de clusterlabels representeren.

---


Hiermee kun je een effectieve clustering van foto's uitvoeren op basis van hun kleurinformatie.



In [ ]:
# Code van ChatGPT

import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Define a function to calculate color histograms
def calculate_histogram(image_path):
    image = cv2.imread(image_path)  # Read the image
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Calculate histograms for each channel
    hist_r = cv2.calcHist([image], [0], None, [256], [0, 256]).flatten()  # Red
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256]).flatten()  # Green
    hist_b = cv2.calcHist([image], [2], None, [256], [0, 256]).flatten()  # Blue

    # Concatenate histograms into a single feature vector
    hist_feature = np.concatenate([hist_r, hist_g, hist_b])
    return hist_feature

### Stap 3: Gebruik DBSCAN voor het detecteren van outlier-afbeeldingen

In deze stap maken we gebruik van **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) om outlier-afbeeldingen te detecteren. Dit algoritme is ideaal voor het identificeren van afwijkingen in datasets, zoals afbeeldingen met kleuren die sterk verschillen van de rest.

---

#### **Benodigde stappen**:

1. **Bereken kleurenhistogrammen met OpenCV**:
   - Gebruik de OpenCV-functie `cv2.calcHist` om de RGB-histogrammen van alle afbeeldingen te berekenen.
   - Combineer de histogrammen van de afzonderlijke kleurkanalen (Rood, Groen, Blauw) tot één enkele featurevector per afbeelding.
   - Deze histogrammen dienen als input voor het clustering-algoritme.

2. **Train DBSCAN op vergelijkbare afbeeldingen**:
   - Selecteer een dataset met afbeeldingen die qua kleur sterk op elkaar lijken (bijvoorbeeld alleen foto's van meren of bossen).
   - Pas DBSCAN toe op de kleurenhistogrammen om de structuur van de data te leren en clusters te vormen.

3. **Test op gemengde afbeeldingen**:
   - Stel een testdataset samen die zowel vergelijkbare afbeeldingen bevat (bijvoorbeeld meren) als afbeeldingen met totaal verschillende kleuren (bijvoorbeeld woestijnen of stadsgezichten).
   - Gebruik het getrainde DBSCAN-model om outliers te identificeren. Afbeeldingen die niet binnen een cluster vallen, worden beschouwd als outliers.

4. **Analyseer de resultaten**:
   - Controleer welke afbeeldingen door DBSCAN als outliers worden gemarkeerd.
   - Evalueer de nauwkeurigheid door te vergelijken of de gemarkeerde outliers daadwerkelijk visueel verschillen van de rest van de dataset.

---

#### **Belangrijke punten**:
- **Instellen van parameters**:
  - DBSCAN heeft twee belangrijke parameters: `eps` (de maximale afstand tussen twee punten in een cluster) en `min_samples` (het minimum aantal punten nodig om een cluster te vormen). Experimenteer met deze parameters om optimale resultaten te krijgen.
- **Gebruik PCA voor dimensiereductie**:
  - Als de kleurenhistogrammen een hoge dimensie hebben, kun je PCA gebruiken om het aantal features te reduceren voordat je DBSCAN toepast.
- **Outliers visualiseren**:
  - Maak gebruik van scatter plots of voorbeelden van afbeeldingen om de outliers visueel te inspecteren.

---

Met DBSCAN kun je effectief outliers in een dataset detecteren, vooral wanneer er afbeeldingen met duidelijke kleurverschillen aanwezig zijn. Het combineren van `cv2.calcHist` met een zorgvuldig samengestelde testdataset maakt dit proces krachtig en inzichtelijk.


## Stap 4: Bouw een autoencoder met RGB waardes. 
Een voorbeeld staat gegeven in Teams (autoencoder_RGB.ipynb)
- Voer de stappen uit. Mochten er dingen niet werken, los deze op. 
- Verklein eventueel de foto's zodat deze sneller runt. Of pak een kleinere dataset. 
- Trainen op deze dataset en 50 epochs, duurde voor mij 15 minuten.




## Stap 5: Bouw een Hybride Autoencoder (Optioneel)

Tot nu toe hebben we gewerkt met een **grayscale autoencoder** of een volledige **RGB autoencoder**. 

Ook  hebben we met behulp van `cv2.calcHist` op een efficiënte manier de kleurinformatie van afbeeldingen verkregen.

De volgende stap is om deze twee benaderingen te combineren. 

Kun je een grayscale autoencoder combineren met kleurhistogramwaarden om een snellere en efficiëntere autoencoder te maken die kleurinformatie verwerkt? 

Dit hybride model kan het beste van beide werelden benutten: de eenvoud van grayscale verwerking en de toegevoegde kracht van kleurinformatie.
